# Torus Knots Counter

This is a SageMath script that computes the motive of the irreducible $\mathrm{SL}_r(k)$-character variety of an $(n,m)$-torus knot, for $r \leq 4$. This is the moduli space of isomorphism classes of irreducible representations
$$
    \rho: \pi_1(S^3 - K_{n,m}) \to \mathrm{SL}_r(k),
$$
where $K_{n,m} \subset S^3$ is the $(n,m)$-torus knot.

The computation is performed by exhausting the possible configurations of eigenvalues $\kappa$ admissible for an irreducible representations and all the types $\tau$ of semi-simple filtrations for a representation with eigenvalues taken from $\kappa$. For further information about the method, please check [1].

*Dependencies:* This script requieres the files 'file_header.txt' and 'file_ending.txt' to be located in the same directory as this script. It also requieres a folder named 'Results' for writing the output.

**Authors:** Ángel González-Prieto (Universidad Politécnica de Madrid) and Vicente Muñoz (Universidad de Málaga).

### References.

[1] Á. González-Prieto and V. Muñoz, *Motive of the $\mathrm{SL}_4(\mathbb{C})$-character variety of torus knots*, 2020.

In [ ]:
%load_ext cython
from sage.all import *

In [ ]:
### CONFIGURATION OF THE FILES WITH THE HEADER
### AND THE ENDING FOR THE GENERATED LATEX FILE
### MODIFY CONSEQUENTLY IF THE PATHS CHANGE

file_header_name = 'file_header.txt'
file_header = open(file_header_name, 'r')
str_header = file_header.read()
file_header.close()

file_ending_name = 'file_ending.txt'
file_ending = open(file_ending_name, 'r')
str_ending = file_ending.read()
file_ending.close()

var('q')

### TYPES GENERATION

'''
    Generates a list of types of reducible semi-simple
    filtrations with eigenvalues configuration taken
    from eigs1 and eigs2.
    
    Each component of the type is a triple (sA, sB, m).
    sA is a list with the eigenvalues of A.
    sB is a list with the eigenvalues of B.
    m is the multiplicity of the isotypic component (sA, sB).
'''
def generate_types(eigs1, eigs2):
    r = len(eigs1)
    types = []
    for p in OrderedPartitions(r): # Steps of the semi-simple filtration
        for s1 in get_ordered_partitions_bilevel(r, p):
            for sigma in SymmetricGroup(r):
                types.append(convert_partition_to_type(s1, eigs1, actList(sigma, eigs2)))
    
    return [typ for typ in types if len(typ) > 1 or len(typ[0]) > 1] # Remove type of irreducible representation

'''
    Compares if two multisets are equal.
'''
def compare_multisets(A, B):
    if len(set(A)) != len(set(B)):
        return False
    
    for a in set(A):
        if (not a in B) or A.count(a) != B.count(a):
            return False
    return True

'''
    Converts a semi-simple representation into its isotypic
    components.
    Warning: Since we are dealing only with isotypic components
    of higher multiplicity of dimension 1, we decree that two
    components are isomorphic iff they have the same eigenvalue (character).
    This no longer holds for higher dimensional representations.
'''
def count_multiplicities_semisimples(semisimple):
    res = []
    
    for eigs_A, eigs_B in semisimple:
        found = False
        for r in res:
            # Warning: Only valid for 1-dimensional representations
            if compare_multisets(r[0], eigs_A) and compare_multisets(r[1], eigs_B):
                r[2] += 1
                found = True
        
        if not found:
            res.append([eigs_A, eigs_B, 1])
    return res

'''
    Converts a multilevel partition to a type
    with eigenvalues taken from eigs1 and eigs2.
'''
def convert_partition_to_type(s, eigs1, eigs2):
    typ = []
    for i in range(len(s)):
        
        semisimples = []
        for j in range(len(s[i])):
            eigs_A = [eigs1[k-1] for k in list(s[i][j])]
            eigs_B = [eigs2[k-1] for k in list(s[i][j])]
            semisimples.append([eigs_A, eigs_B])
    
        typ.append(count_multiplicities_semisimples(semisimples))
    return typ

'''
    Auxiliary function.
'''
def add_multilevel_set_partition(s):
    if len(s) == 0:
        return [[]]
    
    rec_res = add_multilevel_set_partition(s[1:])
    res = []
    for ss in SetPartitions(s[0]):
        for r in rec_res:
            res.append([ss] + r)
    return res

'''
    Auxiliary function.
'''    
def get_ordered_partitions_bilevel(r, p):
    res = []
    for s in OrderedSetPartitions(r,p):
        res = res + add_multilevel_set_partition(s)
        
    return res

'''
    Converts a type to a string. Useful for
    writing the result to a LaTeX file.
'''
def type_to_string(typ):
    str_typ = '('
    str_A = '('
    str_B = '('
    
    for i in range(len(typ)):
        str_typ += '{'
        str_A += '{'
        str_B += '{'
        
        for j in range(len(typ[i])):
            str_typ += '(' + str(len(typ[i][j][0])) + ', '\
            + str(typ[i][j][2]) + ')'
            str_A += '{'
            str_B += '{'
            
            for k in range(len(typ[i][j][0])):
                str_A += '\\epsilon_' + str(typ[i][j][0][k])
                str_B += '\\epsilon_' + str(typ[i][j][1][k])
                if not k == len(typ[i][j][0]) - 1:
                    str_A += ', '
                    str_B += ', '
            str_A +='}'
            str_B += '}'
            
            if not j == len(typ[i])-1:
                str_typ += ', '
                str_A += ', '
                str_B += ', '
                
        str_typ += '}'
        str_A += '}'
        str_B += '}'
        if not (i == len(typ) - 1):
            str_typ += ', '
            str_A += ', '
            str_B += ', '            
    
    return str_typ + ')', str_A + ')', str_B + ')'

'''
    Converts the eigenvalue configuration eigs to
    latex with symbol for the eigenvalues symbol.
'''
def convert_to_latex_eig(eigs, symbol):
    res = '('
    
    for eig in eigs[:-1]:
        res = res + symbol + '_%s'%eig + ','
        
    res = res + symbol + '_%s'%eigs[-1] + ')'
    
    return res

'''
    Auxiliary function for creating the formal variables
    of the number of components of each configuration
    of eigenvalues.
'''
def compute_index_variable_mult(mult1, mult2):
    return sum([mult1[i]*2**i for i in range(len(mult1))]) \
            + sum([mult2[i]*2**(len(mult1) + i) for i in range(len(mult2))])

### E-POLINOMIALS OF SL AND GL

var('q, k')

'''
    Computes the polynomial 1 + q + ... + q^(n-1)
'''
def q_value(n):
    suma = 1
    for k in range(1, n):
        suma += SR('q')**k
    
    return suma

'''
    Computes the q-factorial.
'''
def q_factorial(n):
    if n == 1:
        return q_value(1)
    
    return q_value(n)*q_factorial(n-1)

'''
    Returns the motive of SLr, where q is the
    Lefschetz motive (the virtual class of the affine line).
'''
def get_SLr(n):
    if n == 1:
        return 1
    else:
        return (SR('q')**(binomial(n,2))*(SR('q')-1)**(n-1)*q_factorial(n)).factor()

'''
    Returns the motive of GLr, where q is the
    Lefschetz motive (the virtual class of the affine line).
'''
def get_GLr(n):
    return get_SLr(n)*(SR('q')-1)


### LIST AND BLOCKS HANDLING

def convert_part_to_list(p):
    result = []
    
    for i in range(len(p)):
        result = result + [i+1]*p[i]
    return result

'''
    Permutes the indices of the elements of l
    according to the permutation s.
'''
def actList(s, l):
    return [l[s(i + 1) - 1] for i in range(len(l))]

'''
    Permutes the elements of a list of integers l
    according to the permutation s.
'''
def actListElementwise(s, l):
    return [s(elem) for elem in l]
    
'''
    Converts a collection of eigenvalues to
    a canonical form i.e. with eigenvalues
    enumerated from 1, 2, ..., n.
'''
def convert_to_canonical(eig):
    canon = 1
    orig = 1
    
    eig_conv = copy(eig)
    eig_conv.sort()
    
    i = 0
    while i < len(eig):
        while not orig in eig_conv:
            orig = orig + 1
        
        while i < len(eig) and eig_conv[i] == orig:
            eig_conv[i] = canon
            i = i + 1
            
        orig = orig + 1
        canon = canon + 1
    
    return eig_conv
        
### COMPARATION AND GENERATION OF TYPES

'''
    Generates all the possible classes of types with configuration
    of eigenvalues epsilon = eig1 and varepsilon = eig2
    and counts their multiplicity.
'''
def generate_types_multiplicities(eig1, eig2):
    types_multiplicities = []
    
    for t in generate_types(eig1, eig2):
        update_list_types(t, types_multiplicities)

    return [[types_multiplicities[i][0], len(types_multiplicities[i][1])]\
            for i in range(len(types_multiplicities))]

'''
    Extracts the multiplicities of all the eigenvalues
    of the type typ.
'''
def extract_eigenvalues_mult(typ):
    eigs_A = {}
    eigs_B = {}
    
    for i in range(len(typ)):
        for j in range(len(typ[i])):
            for eig_A in typ[i][j][0]:
                if eig_A in eigs_A:
                    eigs_A[eig_A] += typ[i][j][2]
                else:
                    eigs_A[eig_A] = typ[i][j][2]
                    
            for eig_B in typ[i][j][1]:
                if eig_B in eigs_B:
                    eigs_B[eig_B] += typ[i][j][2]
                else:
                    eigs_B[eig_B] = typ[i][j][2]
                    
    return eigs_A, eigs_B

'''
    Checks whether the permutation s preserves the
    multiplicities of the multiset eigs or not.
'''                 
def permutation_preserve_mult(s, eigs):
    for eig in eigs.keys():
        if (not (s(eig) in eigs)) or eigs[s(eig)] != eigs[eig]:
            return False
        
    return True

'''
    Computes the action of two permutations s_A and s_B on
    the type typ. The permutation s_A permutes the eigenvalues
    of typ corresponding to the matrix A and s_B permutates
    the eigenvalues corresponding to B.
'''
def act_type(s_A, s_B, typ):
    res_typ = deepcopy(typ)
    
    for i in range(len(res_typ)):
        for j in range(len(res_typ[i])):
            res_typ[i][j][0] = actListElementwise(s_A, res_typ[i][j][0])
            res_typ[i][j][1] = actListElementwise(s_B, res_typ[i][j][1])
    
    return res_typ

'''
    Computes whether the types typ_1 and typ_2 are equivalent or not.
'''
def are_equivalent_types(typ_1, typ_2):
    eigs_A_1, eigs_B_1 = extract_eigenvalues_mult(typ_1)
    eigs_A_2, eigs_B_2 = extract_eigenvalues_mult(typ_2)

    n_A = max(list(eigs_A_1.keys()) + list(eigs_A_2.keys()))
    n_B = max(list(eigs_B_1.keys()) + list(eigs_B_2.keys()))
    
    for s_A in SymmetricGroup(n_A):
        if permutation_preserve_mult(s_A, eigs_A_1):
            for s_B in SymmetricGroup(n_B):
                if permutation_preserve_mult(s_B, eigs_B_1):
                    if compare_types(act_type(s_A, s_B, typ_1), typ_2):
                        return True
                    
    return False

'''
    Compares whether the semi-simple representations
    ss_1 and ss_2 are isomorphic or not.
'''
def compare_semisimples(ss_1, ss_2):
    if len(ss_1) != len(ss_2):
        return False
    
    new_ss1 = copy(ss_1)
    new_ss2 = copy(ss_2)

    while len(new_ss1) > 0:
        irr1 = new_ss1.pop()
        
        found = False
        for irr2 in new_ss2:
            if not found and set(irr1[0]) == set(irr2[0]) \
            and set(irr1[1]) == set(irr2[1]) and irr1[2] == irr2[2]:
                found = True
                new_ss2.remove(irr2)
                
        if not found:
            return False

    return True

'''
    Compares whether the types typ_1 and typ_2 are
    equal or not.
'''
def compare_types(typ_1, typ_2):
    if len(typ_1) != len(typ_2):
        return False
    
    for i in range(len(typ_1)):
        if not compare_semisimples(typ_1[i], typ_2[i]):
            return False
    return True

'''
    Checks whether the type typ is in the list l
'''
def type_is_in_list(typ, l):
    for typ_p in l:
        if compare_types(typ, typ_p):
            return True
    return False

'''
    Updates the list of types l with the type typ.
    If typ is equivalent to a type of l, the function
    adds typ to the sublist of equivalent types.
    If typ is not equivalent to any element of l,
    the function adds typ to l.
'''
def update_list_types(typ, l):
    found = False
    
    i = 0
    while not found and i < len(l):
        typ_i = l[i][0]
        # Is equivalent to an element of the list
        if are_equivalent_types(typ, typ_i):
            found = True
            if not type_is_in_list(typ, l[i][1]):
                # Is not isomorphic to any element of the sublist
                # -> increments multiplicity.
                l[i][1].append(typ)
        i += 1
            
    if not found:
        l.append([typ, [typ]])
    
    return

'''
    Computes the rank of the representations of type typ.
'''
def total_len_type(typ):
    leng = 0
    for i in range(len(typ)):
        for j in range(len(typ[i])):
            leng += len(typ[i][j][0])*typ[i][j][2]
    return leng
                   
### PARAMETER COUNTING FUNCTIONS FOR MATRICES

'''
    Computes the motive of R(tau) for the the type tau = typ.
    Multiplies the result by the multiplicity of tau given by mult.
    Returns a tuple (R_typ_components, str_out), where
    R_typ_components is the value of the motive mult*[R(tau)]
    and str_out is a string with the information of the computation.
'''
def compute_Rtau(typ, mult):
    str_out = ''
    
    # [Mtau]
    Mtau = SR(compute_Mtau(typ))
    Rtau = Mtau
    if Rtau == 0:
        str_out += 'Not contributes (Mtau = 0)\n'
        return SR(0), str_out

    # Irreducible part ({\frak M}tau)
    irred_part = compute_semisimplification(typ)            
    if irred_part == 0:
        str_out += 'Not contributes (Semisimplification = 0)\n'
        return SR(0), str_out
    
    # Gauge group Gtau
    stab = compute_gauge(typ)    
    if stab is None:
        str_out += 'Not contributes (gauge = 0)\n'
        return SR(0), str_out
    else:
        Rtau = Rtau/stab

        
    R_typ = Rtau * irred_part * get_GLr(total_len_type(typ))
    R_typ_components = mult * R_typ
    
    str_out += '\\begin{itemize}\n'
    str_out += '\\item $[\\mathcal{M}_{\\tau}] = ' + latex(Mtau) + '.$\n\n'
    str_out += '\\item $[\\mathcal{G}_{\\tau}] = ' + latex(stab) + '.$\n\n'
    str_out += '\\item $[{\\frak M}_{\\tau}^{\\textrm{irr}}] = ' + latex(irred_part.simplify_full()) + '.$\n\n'
    str_out += '\\item $[R(\\tau)] =' + latex(R_typ.simplify_full()) + '$\n\n'
    str_out += '\\item $m_{\\kappa}(\\tau) = ' + latex(mult) + '.$\n\n'   
    str_out += '\\end{itemize}\n'
    
    return R_typ_components, str_out

'''
    Computes the integer Cij (useful for computing [Mtau])
'''
def compute_Cij(typ, i, j):
    Cij = 0
    for l in range(len(typ[i])):
        Cij += typ[i][l][2]*typ[i+1][j][2]*(len(typ[i][l][0])*len(typ[i+1][j][0])\
                                            - compute_equals_eigenvalues_B(typ, i, l, i+1, j))
    
    return Cij

'''
    Computes the integer C (useful for computing [Mtau])
'''
def compute_C(typ):
    C = 0
    
    for i in range(len(typ)):
        for ip in range(i-1):
            for j in range(len(typ[i])):
                for jp in range(len(typ[ip])):
                    C+= typ[i][j][2]*typ[ip][jp][2]*(len(typ[i][j][0])*len(typ[ip][jp][0])\
                                                     - compute_equals_eigenvalues_B(typ, i, j, ip, jp))
    return C

'''
    Computes the integer Dij (useful for computing [Mtau])
'''
def compute_Dij(typ, i, j):
    Dij = 0
    
    for l in range(len(typ[i])):
        Dij += compute_equals_eigenvalues_A(typ, i, l, i+1, j)
        if set(typ[i][l][0]) == set(typ[i+1][j][0]) and set(typ[i][l][1]) == set(typ[i+1][j][1]):
            Dij -= 1
    return Dij

'''
    Computes the motive [Mtau] of the parameters
'''
def compute_Mtau(typ):
    acc = 1
    for i in range(0, len(typ)-1):
        for j in range(len(typ[i+1])):
            # m_{i+1,j} > 1 and the previous step contains
            # only a 1-dimensional representation
            if len(typ[i+1][j][0]) == 1 and typ[i+1][j][2] > 1 and len(typ[i]) == 1:
                return SR(0)
            
            # m_{i+1,j} = 2 and the previous step is made of two non-isomorphic
            # 1-dimensional representations
            elif len(typ[i+1][j][0]) == 1 and typ[i+1][j][2] == 2 and len(typ[i]) == 2:
                Cij = compute_Cij(typ, i, j)
                acc = acc*(q^Cij - (q^3 + q^2 - q))
            
            # The isotypic component W_{i+1,j} has multiplicity 1 (m_{i+1,j} = 1)
            elif typ[i+1][j][2] == 1:
                Cij = compute_Cij(typ, i, j)
                Dij = compute_Dij(typ, i, j)
                acc = acc*(q^Cij - q^Dij)
            else:
                print('WARNING: Not implemented type: ' + str(typ))
                return SR(0)
    
    # Contribution of non-adjacent steps
    C = compute_C(typ)
    
    if C == 0:
        return acc
    else:
        return q^C*acc

'''
    Computes the motive of the semisimplification part,
    {\frak M}tau.
'''
def compute_semisimplification(typ):
    irred_part = 1
    for i in range(len(typ)):
        for j in range(len(typ[i])):            
            irred_part = irred_part*get_motive_Rirred_kappa(typ[i][j][0], typ[i][j][1])\
            /get_SLr(len(typ[i][j][0]))
    
    return irred_part

'''
    Auxiliary function.
'''
def count_repetitions(eig_alpha, eig_beta):
    suma = 0
    
    for eig in set(eig_alpha):
        if eig in eig_beta:
            suma += eig_alpha.count(eig)*eig_beta.count(eig)
    return suma

'''
    Auxiliary function.
'''
def compute_equals_eigenvalues_A(typ, i, j, l, h):
    A_alpha = typ[i][j][0]
    A_beta = typ[l][h][0]
    
    return count_repetitions(A_alpha, A_beta)*typ[i][j][2]*typ[l][h][2]

'''
    Auxiliary function.
'''    
def compute_equals_eigenvalues_B(typ, i, j, l, h):
    B_alpha = typ[i][j][1]
    B_beta = typ[l][h][1]
    
    return count_repetitions(B_alpha, B_beta)*typ[i][j][2]*typ[l][h][2]

'''
    Computes the motive of the gauge group, Gtau.
'''
def compute_gauge(typ):
    acc = 1
    
    for i in range(len(typ)):
        for j in range(len(typ[i])):
            acc *= get_GLr(typ[i][j][2])
    
    D = 0 
    for i in range(len(typ)):
        for l in range(len(typ[i])):
            for j in range(i):
                for h in range(len(typ[j])):
                    D += compute_equals_eigenvalues_A(typ, i, l, j, h)
    
    if D == 0:
        return acc
    else:
        return q^D*acc

'''
    Auxiliary function.
'''   
def get_multiplicities(eigenvalues):
    eigs = {}
    
    for eig in eigenvalues:
        if not eig in eigs:
            eigs[eig] = 1
        else:
            eigs[eig] = eigs[eig] + 1
            
    return eigs

'''
    Computes whether all the representations
    with configuration of eigenvalues eig1 and eig2
    are reducible since they are forced to share
    an eigenvalue for dimensional reasons.
'''
def are_all_reducible(eig1, eig2):
    r = len(eig1)
    
    if r <= 1:
        return False
    
    eig1_can = convert_to_canonical(eig1)
    eig2_can = convert_to_canonical(eig2)
    
    mult1 = get_multiplicities(eig1_can)
    mult2 = get_multiplicities(eig2_can)
    
    larg1 = max(mult1.values())
    larg2 = max(mult2.values())
        
    if larg1 + larg2 > r:
        return True
    else:
        return False
    
### TOP FUNCTIONS

# Remark: The case [1, 1, 2, 2],[1, 1, 2, 2] has always
# a 2-dimensional invariant subspace
epols = [([1], [1], SR(1)), ([1, 1, 2, 2],[1, 1, 2, 2], SR(0))]

'''
    Returns the motive [Rirred_kappa] of irreducible representations
    for a fixed configuration of eigenvalues kappa = (eig1, eig2).
    Lazy calculation: if it was previously computed, returns the
    value without re-computing it.
'''
def get_motive_Rirred_kappa(eig1, eig2, verbose = False, out_file = None):
    eig1_can = convert_to_canonical(eig1)
    eig2_can = convert_to_canonical(eig2)
    
    if are_all_reducible(eig1_can, eig2_can):
        return SR(0)
    
    for e1, e2, p in epols:
        if (e1 == eig1_can and e2 == eig2_can):
            return p
    
    final = (compute_motive_Rirred_kappa(eig1, eig2, verbose, out_file)).simplify_full()
    if verbose:
        print('*****')
        print('New polynomial')
        show(eig1)
        show(eig2)
        show(final)
        print('*****')
    epols.append((eig1, eig2, final))
    return final

'''
    Computes the motive [R_kappa] for a fixed
    configuration of eigenvalues kappa = (eig1, eig2).
'''
def compute_motive_total(eig):
    numer = get_GLr(len(eig))
    
    eig_sort = copy(eig)
    eig_sort.sort()
    
    denom = 1
    i = 0
    while i < len(eig_sort):
        block_size = 1
        eig_block = eig_sort[i]
        i = i + 1
        while i < len(eig_sort) and eig_sort[i] == eig_block:
            block_size = block_size + 1
            i = i + 1
        
        denom = denom * get_GLr(block_size)
    
    denom = denom
    
    return numer/denom
            
'''
    Computes the motive [Rred_kappa] of reducible representations
    for a fixed configuration of eigenvalues kappa = (eig1, eig2).
'''
def compute_motive_Rred_kappa(eig1, eig2, verbose = False,\
                             out_file = None, only_id_blocks = False):
    flags = generate_types_multiplicities(eig1, eig2)
    r = len(eig1)

    total = 0
    cont = 1

    for shape, mult in flags:
        
        params, str_out = compute_Rtau(shape, mult)
        params = params.simplify_full()
        
        if params != 0 and not out_file is None \
            and (not only_id_blocks or len(get_full_id_blocks(E1, E2)) > 0):
            
            str_typ, str_A, str_B = type_to_string(shape)
            out_file.write('\\noindent\\rule{8cm}{0.4pt}\n\n')
            out_file.write('$$\\xi = '+ str_typ + ',\\quad \\sigma_A = ' + str_A \
                           + ',\\quad \\sigma_B = ' + str_B + '.$$\n\n')
    
            out_file.write(str_out)
        
        if verbose:
            print('++++++++++')
            print(cont)
            cont = cont + 1
            print(shape)
            print(params)
            
        total = total + params

    Ered = SR(total).simplify_full()    
    return Ered

'''
    Computes the motive [Rred_kappa] of reducible representations
    for a fixed configuration of eigenvalues kappa = (eig1, eig2).
    In contrast with get_motive_Rirred_kappa, this function
    performs the effective calculation.
'''
def compute_motive_Rirred_kappa(eig1, eig2, verbose = False,\
                       out_file = None, only_id_blocks = False):  
    r = len(eig2)
    
    total1 = compute_motive_total(eig1)
    total2 = compute_motive_total(eig2)
    Etotal = total1*total2
    
    resultado = 0        
    Ered = compute_motive_Rred_kappa(eig1, eig2, verbose, out_file, only_id_blocks)
           
    if verbose:
        print('---')
        print('Etotal')
        show(Etotal.simplify_full())
        print('Ered')
        show(Ered.simplify_full())
                
    resultado = Ered
    
    return (Etotal - resultado).simplify_full()    
    
'''
    Computes the motive [Rirr_r] of reducible representations
    of rank r and saves the result to the file out_file.
'''    
def compute_motive_Rirred(r, verbose = False, out_file = None):
    resultado = 0
    resultado_quot = 0
    
    e_pols = []
    counter = 0
    for p in Partitions(r):
        eig1 = convert_part_to_list(p)
        for pp in Partitions(r):
            eig2 = convert_part_to_list(pp)
            
            print('---- PARTITION: %s/%s'%(counter, len(Partitions(r))**2))
            counter += 1
            
            if not are_all_reducible(eig1, eig2):
                if not (out_file is None):
                    out_file.write('\\section{Configuration ')
                    out_file.write('$\\bm{\\epsilon} = ' \
                                   + convert_to_latex_eig(eig1, '\\epsilon') + '$ and ')
                    out_file.write('$\\bm{\\varepsilon} =' \
                                   + convert_to_latex_eig(eig2, '\\varepsilon') + '$')
                    out_file.write('}\n')
                
                l = get_motive_Rirred_kappa(eig1, eig2, verbose = verbose,\
                                   out_file = out_file).simplify_full()
                
                eig1_can = convert_to_canonical(eig1)
                eig2_can = convert_to_canonical(eig2)
    
                mult1 = list(get_multiplicities(eig1_can).values())
                mult2 = list(get_multiplicities(eig2_can).values())
                
                mult1.sort()
                mult2.sort()
            
                c = var('C_%s'%(compute_index_variable_mult(mult1, mult2)),\
                        latex_name = 'C_{'+ str(tuple(mult1)) + ', ' + str(tuple(mult2)) +'}')
                resultado = (resultado + SR(c)*l).collect(c)
                resultado_quot = (resultado_quot + SR(c)*(l/get_SLr(r)).simplify_full()).collect(c)

                
                if not out_file is None:
                    total1 = compute_motive_total(eig1)
                    total2 = compute_motive_total(eig2)
                    Etotal = total1*total2    
                    out_file.write('\\noindent\\rule{9cm}{2pt}\\vspace{0.2cm}\n\n'+\
                                   '\\noindent\\textbf{Total count of $\\kappa = ('\
                                   + convert_to_latex_eig(eig1, '\\epsilon') +', '\
                                   + convert_to_latex_eig(eig2, '\\varepsilon') + ')$}\\medskip\n\n')
                    out_file.write('${[R_{\\kappa}^{\\textrm{red}}]} = ' \
                                   +  latex((Etotal - l).simplify_full()) + ',$\n\n')
                    out_file.write('${[R_{\\kappa}^{\\textrm{irr}}]} = ' \
                                   + latex(l) + ',$\n\n')
                    out_file.write('${[R_{\\kappa}]} = ' + latex(Etotal.simplify_full()) + ',$\n\n')
                    out_file.write('${[{\\frak M}_{\\kappa}]} = ' \
                                   + latex((l/get_SLr(r)).simplify_full()) + '.$\n\n\\newpage{}\n\n')
                
                if verbose:
                    print('+*+*+*+*+*+*+*+*+*')
                    print(eig1)
                    print(eig2)
                    show(l)
                    show((l/get_SLr(r)).simplify_full())
                    show(resultado)
                
                e_pols.append((eig1, eig2, l))
            elif verbose:
                print('+*+*+*+*+*+*+*+*+*')
                print('All reducibles')
                print(eig1)
                print(eig2)
    
    if not out_file is None:
        out_file.write('\\section*{Summary}\n')        
        
        for eig1, eig2, l in e_pols:
            out_file.write('$[R_{' + convert_to_latex_eig(eig1, '\\epsilon') \
                           + ',' + convert_to_latex_eig(eig2, '\\varepsilon') \
                           + '}^{\\textrm{irr}}] = ' + latex(l) + '.$\n\n')
        
        str_resultado = latex(resultado).replace('{\\left(', '(').replace('\\right)}', ')')
        str_resultado_quot = latex(resultado_quot).replace('{\\left(', '(').replace('\\right)}', ')')
        
        out_file.write('\n\\medskip\\textbf{Final result representations}.\\medskip\n\n')
        out_file.write('$[R_{' + str(r) + '}^{\\textrm{irr}}] = ' + str_resultado + '.$\n')
        out_file.write('\n\\medskip\\textbf{Final result characters}.\\medskip\n\n')
        out_file.write('$[{\\frak M}_{' + str(r) + '}^{\\textrm{irr}}] = ' \
                       + str_resultado_quot + '.$\n')

        
    return resultado.simplify_full()

'''
    TOP FUNCTION
    Computes the motive [Rirr_r] of reducible representations
    of rank r. Creates a LaTeX file of name file_name and
    saves the result there.
'''    
def compute_motive_Rirred_file(r, file_name):
    out_file = open(file_name, 'w')
    out_file.write(str_header.replace('&rank&', '{' + str(r) + '}'))
    
    res = compute_motive_Rirred(r, verbose=False, out_file = out_file)
    out_file.write(str_ending)
    out_file.close()
    
    return res

In [ ]:
r = 2
file_name = './Results/rank' + str(r) + '.tex'
compute_motive_Rirred_file(r, file_name)